In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F

custom_transform = transforms.Compose([
    transforms.RandomResizedCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

training_data = datasets.Flowers102(
    root="data",
    split= "train",
    download=True,
    transform= custom_transform,
)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

test_data = datasets.Flowers102(
    root="data",
    split= "test",
    download=True,
    transform= custom_transform,
)

test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

count=0
for i, (X_Train, y_train) in enumerate(training_data):
  count+= 1

conv1 = nn.Conv2d(3, 18, 3, 1, 'same')
conv2 = nn.Conv2d(18, 144, 3, 1, 'same')
conv3 = nn.Conv2d(144, 360, 3, 1, 'same')
x = F.relu(conv1(X_Train))

x = F.max_pool2d(x, 2, 2)
x.shape

x = F.relu(conv2(x))

x = F.max_pool2d(x, 2, 2)
x = F.relu(conv3(x))

x = F.max_pool2d(x, 2, 2)
x.shape

 99%|█████████▊| 339738624/344862509 [00:20<00:02, 1920225.87it/s]

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(3, 9, 3, 1, 'same')
        self.conv2 = nn.Conv2d(9, 18, 3, 1, 'same')
        #Fully Connected Layer
        self.fc1 = nn.Linear(18*56*56, 200)
        self.fc2 = nn.Linear(200, 150)
        self.fc3 = nn.Linear(150, 102)

    def forward(self, X):
      x = self.flatten(X)
      X = F.relu(self.conv1(X))
      X = F.max_pool2d(X,2,2)
      #Second Pass
      X = F.relu(self.conv2(X))
      X = F.max_pool2d(X,2,2)
      #Fully Connected Layers
      X = X.view(-1, 18*56*56)
      X = F.relu(self.fc1(X))
      X = F.relu(self.fc2(X))
      X = self.fc3(X)
      return F.log_softmax(X, dim=1)

In [ ]:
torch.manual_seed(64)
model  =  NeuralNetwork()
model

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if batch % 1 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    test_loss = 0
    correct = 0
    num_batches = len(dataloader)
    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            _, predicted = torch.max(pred, 1)
            correct += (predicted == y).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time
start = time.time()

batch_size = 64
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

end = time.time()
total = end - start
print(f"Training took: {total/60} minutes!")

Epoch 1
-------------------------------
loss: 4.632421  [   64/ 1020]
loss: 4.728790  [  128/ 1020]
loss: 4.807528  [  192/ 1020]
loss: 4.737648  [  256/ 1020]
loss: 4.646730  [  320/ 1020]
loss: 4.665675  [  384/ 1020]
loss: 4.775331  [  448/ 1020]
loss: 4.666566  [  512/ 1020]
loss: 4.619850  [  576/ 1020]
loss: 4.611726  [  640/ 1020]
loss: 4.597502  [  704/ 1020]
loss: 4.623504  [  768/ 1020]
loss: 4.622926  [  832/ 1020]
loss: 4.589792  [  896/ 1020]
loss: 4.602981  [  960/ 1020]
loss: 4.629015  [ 1020/ 1020]
Test Error: 
 Accuracy: 2.1%, Avg loss: 4.585814 

Epoch 2
-------------------------------
loss: 4.541603  [   64/ 1020]
loss: 4.549908  [  128/ 1020]
loss: 4.526780  [  192/ 1020]
loss: 4.540767  [  256/ 1020]
loss: 4.548572  [  320/ 1020]
loss: 4.470823  [  384/ 1020]
loss: 4.534067  [  448/ 1020]
loss: 4.357556  [  512/ 1020]
loss: 4.527840  [  576/ 1020]
loss: 4.304918  [  640/ 1020]
loss: 4.374347  [  704/ 1020]
loss: 4.496682  [  768/ 1020]
loss: 4.436764  [  832/ 1020]